In [1]:
import numpy as np
%cd  '/home/ubuntu/image-drift-monitoring/data/camelyon17_v1.0'
%pwd

camelyon_train_comp = np.load('camelyon_train_ds.npz')

/home/ubuntu/image-drift-monitoring/data/camelyon17_v1.0


In [2]:
camelyon_train = camelyon_train_comp['arr_0']

np.random.shuffle(camelyon_train)

In [3]:
camelyon_train_0_25   = camelyon_train[:int(len(camelyon_train)/4)]
camelyon_train_25_50  = camelyon_train[int(len(camelyon_train)/4):int(len(camelyon_train)/2)]
camelyon_train_0_12_5  = camelyon_train_0_25[:int(len(camelyon_train_0_25)/2)]
camelyon_train_12_5_25 = camelyon_train_0_25[int(len(camelyon_train_0_25)/2):]

In [4]:
import json 
with open('/home/ubuntu/image-drift-monitoring/config/common/drift_detection_config.json') as config_file:
        drift_detection_config = json.load(config_file)

In [5]:
%cd  '/home/ubuntu/image-drift-monitoring'
%pwd
from src.modules.alibi_detect.untrained_encoder import UntrainedAutoencoder

myUAE = UntrainedAutoencoder(drift_detection_config)

/home/ubuntu/image-drift-monitoring


2023-02-09 13:46:54.692338: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-09 13:46:54.692409: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/ubuntu/miniconda3/envs/alibitorchwhy/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
myUAE.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train_0_12_5.shape[1:])
myUAE.init_detector(detector_type='KS',reference_data=camelyon_train_0_12_5,detector_name='UAE_12_5_P_camelyon_train_KS',save_dec=True)

2023-02-09 13:46:58.389294: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-09 13:46:58.389351: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-09 13:46:58.389370: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (render): /proc/driver/nvidia/version does not exist
2023-02-09 13:46:58.389577: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-09 13:46:58,453 [INFO]  KS Detector initialized
Directory /home/ubuntu/image-drift-monitoring/config/detectors/UA

# H0 Hypothesis testing

In [7]:
myUAE.make_prediction(target_data=camelyon_train_12_5_25,detector_type='KS')

Drift? No!
Feature-wise p-values:
[0.33044982 0.43880793 0.533961   ... 0.17318937 0.8497759  0.8227    ]
len:27648


{'data': {'is_drift': 0,
  'distance': array([0.00688355, 0.00630259, 0.00585571, ..., 0.00803472, 0.00443333,
         0.00457132], dtype=float32),
  'p_val': array([0.33044982, 0.43880793, 0.533961  , ..., 0.17318937, 0.8497759 ,
         0.8227    ], dtype=float32),
  'threshold': 1.8084490740740741e-06},
 'meta': {'name': 'KSDrift',
  'online': False,
  'data_type': None,
  'version': '0.10.4',
  'detector_type': 'drift'}}

# drift testing

In [8]:
camelyon_test_comp = np.load('/home/ubuntu/image-drift-monitoring/data/camelyon17_v1.0/camelyon_test_ds.npz')
camelyon_test = camelyon_test_comp['arr_0']

np.random.shuffle(camelyon_test)

In [9]:
myUAE.make_prediction(target_data=camelyon_test,detector_type='KS')

Drift? Yes!
Feature-wise p-values:
[0. 0. 0. ... 0. 0. 0.]
len:27648


{'data': {'is_drift': 1,
  'distance': array([0.14553557, 0.14744143, 0.14889838, ..., 0.08499561, 0.08281262,
         0.08471379], dtype=float32),
  'p_val': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
  'threshold': 1.8084490740740741e-06},
 'meta': {'name': 'KSDrift',
  'online': False,
  'data_type': None,
  'version': '0.10.4',
  'detector_type': 'drift'}}